In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
import os
from jilm.model import JILMLangModel


gptj_model_load: loading model from '/mnt/c/Users/jeanb/models/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 4505.45 MB
gptj_model_load: memory_size =   896.00 MB, n_mem = 57344
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [2]:
from jilm.document_loader import DocumentLoader
from jilm.settings import CHROMA_SETTINGS, PERSIST_DIRECTORY

In [3]:
embeddings_model_name = os.environ.get('EMBEDDINGS_MODEL_NAME')
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)


ValueError: Could not import sentence_transformers python package. Please install it with `pip install sentence_transformers`.

In [ ]:
doc = DocumentLoader.load_single_document("../README.md")


In [ ]:
# Create and store locally vectorstore
db = Chroma.from_documents([doc], embeddings, persist_directory=PERSIST_DIRECTORY, client_settings=CHROMA_SETTINGS)
db.persist()
db = None

In [ ]:
retriever = db.as_retriever()

In [ ]:
#llm = JILMLangModel(retriever=retriever, embeddings=embeddings, max_tokens=1000, chunk_size=64, chunk_overlap=0)
llm = JILMLangModel(callbacks=[StreamingStdOutCallbackHandler()], retriever=retriever, embeddings=embeddings, max_tokens=1000, chunk_size=64, chunk_overlap=0)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [ ]:
query = "What is the name of the project?"
res = qa(query)    
answer, docs = res['result'], res['source_documents']

In [ ]:
for document in docs:
    print("\n> " + document.metadata["source"] + ":")
    print(document.page_content)